In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

In [2]:
# POINT TO TARGET SITE 

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# PULL DATA FROM SITE

html_doc = requests.get(url).text
#html_doc

In [4]:
# CREATE BEAUTIFUL SOUP OBJECT 

from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

#print(soup.prettify())

In [5]:
# INITIALIZE 4 LISTS TO COLLECT DATA

a0 = []
a1 = []
a2 = []
a3 = []

In [6]:
# PARSE DATA AND STORE IN LISTS 

n = 0
for string in soup.table.stripped_strings:

    if(n==3):
        #print("n=3")
        entry = repr(string)
        entry.strip()
        #print(entry)
        a3.append(entry)
        n=0
    
    if(n==2):
        #print("n=2")
        entry = repr(string)
        entry.strip()
        #print(entry)
        a2.append(entry)
        n=3

    if(n==1):
        #print("n=1")
        entry = repr(string)
        entry.strip()
        #print(entry)
        a1.append(entry)
        n=2  

    if(n==0):
        #print("n=0")
        entry = repr(string)
        entry.strip()
        #print(entry)
        a0.append(entry)
        n=1

In [7]:
# EXTRACT RELEVANT LISTS AND PUT INTO DF
df0 = pd.DataFrame(a0)
df1 = pd.DataFrame(a1)
df2 = pd.DataFrame(a2)

In [8]:
df_final = pd.concat([df0, df1, df2], axis=1)

In [9]:
df_final.head()

,0,0,0
0,'Postcode','Borough','Neighbourhood'
1,'M1A','Not assigned','Not assigned'
2,'M2A','Not assigned','Not assigned'
3,'M3A','North York','Parkwoods'
4,'M4A','North York','Victoria Village'


In [10]:
# MAKE 1ST ROW THE HEADER 

df_final.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_final = df_final[1:]

In [11]:
# REMOVE SPECIAL CHARACTERS 

df_final["Borough"] = df_final["Borough"].str.replace("\'", '')
df_final["Postcode"] = df_final["Postcode"].str.replace("\'", '')
df_final["Neighbourhood"] = df_final["Neighbourhood"].str.replace("\'", '')

In [12]:
df_final.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [13]:
# REMOVE ALL BOROUGHS WHICH HAVE "NOT ASSIGNED"
df_final2 = df_final.query('Borough != "Not assigned"').copy()
df_final2.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [14]:
# REPLACE ANY NOT ASSIGNED OCCURENCES IN NEIGHBOURHOOD WITH BOROUGH NAME
for index, row in df_final2.iterrows():
    if row['Neighbourhood'] == "Not assigned":
        row['Neighbourhood'] = row['Borough']

In [15]:
# CHECK
df_final2.query('Borough == "Not assigned"')

,Postcode,Borough,Neighbourhood


In [16]:
# COMBINE NEIGHBOURHOOD IF POSTCODE SAME
df_final3 = df_final2.groupby(['Postcode','Borough'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ', '.join(x))

In [17]:
df_final3.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [18]:
df_final3.shape

(103, 3)

# Part 2

In [22]:
# DOWNLOAD GEO LOCATION DATA
!wget -O geo.csv https://cocl.us/Geospatial_data

--2018-12-25 11:35:48--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-12-25 11:35:50--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-12-25 11:35:50--  https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.ent.box.com (ibm.ent.box.com)... 107.152.26.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|107.152.26.211|:443... connected.
HTTP request sent, awaiting response... 302 Found

In [25]:
#IMPORT GEOLOCATION DATA
df_geo = pd.read_csv("geo.csv")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
# RENAME HEADER 
df_geo.rename(columns = {"Postal Code":"Postcode"}, inplace = True)
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
df_withGeo = pd.merge(df_final3, df_geo, how='left', on=['Postcode'])
df_withGeo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
